## Fundamentei

This script reads a few stocks infos available in https://fundamentei.com/ using scrapers.

In [ ]:
import os
import pandas
import sys

from tenacity import retry, wait_random, stop_after_attempt
from tqdm import tqdm

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.scrapers.fundamentei import (
    StockInfosScraper
)

### Reading stock infos

In [2]:
# reading fundamentus tickers list
fundamentus_tickers = pandas.read_csv("../outputs/fundamentus.csv", sep=";")["Papel"]
fundamentus_tickers.columns = ['stock_ticker']

# reading status invest tickers list
statusinvest_tickers = pandas.read_csv("../outputs/statusinvest.csv", sep=";")["TICKER"]
statusinvest_tickers.columns = ['stock_ticker']

In [3]:
# get deduped tickers list
stock_tickers = pandas.concat([fundamentus_tickers, statusinvest_tickers]).drop_duplicates().values

In [4]:
@retry(stop=stop_after_attempt(5), wait=wait_random(min=10, max=20))
def get_stock_infos_df(stock_ticker: str) -> pandas.DataFrame:
    return StockInfosScraper(stock_ticker).dataframe_format

In [5]:
# get info from all stocks
stock_infos_df = None
failed_tickers = []

for stock_ticker in tqdm(stock_tickers):
    try:
        # read stock infos
        si_df = get_stock_infos_df(stock_ticker)
        si_df.columns = [stock_ticker]
        si_df = si_df.T
        stock_infos_df = pandas.concat([stock_infos_df, si_df])
    except:
        failed_tickers.append(stock_ticker)

# show company infos retrieved from Fundamentei
print(stock_infos_df[stock_infos_df.columns[0:3]])
print(f"Filed stock tickers: {failed_tickers}")


100%|██████████| 220/220 [14:28<00:00,  3.95s/it]

      Ticker  Nome da Empresa Nota dos Usuários
VALE3  VALE3             Vale               4.3
PETR4  PETR4        Petrobras               3.7
ITUB4  ITUB4       Banco Itaú               4.8
BBDC4  BBDC4   Banco Bradesco               4.3
BBAS3  BBAS3  Banco do Brasil               4.7
...      ...              ...               ...
EUCA4  EUCA4          Eucatex               2.7
CAMB3  CAMB3          Cambuci               3.9
ELMD3  ELMD3      Eletromidia               2.6
LAND3  LAND3      Terra Santa               3.8
FIQE3  FIQE3         Unifique               4.4

[213 rows x 3 columns]
Filed stock tickers: ['RCSL3', 'BOAS3', 'SQIA3', 'ENBR3', 'MODL3', 'PARD3', 'BRPR3']


### Output result into a file   

In [6]:
stock_infos_df.to_csv("../outputs/fundamentei.csv", sep=";", index=False)